In [1]:
import numpy as np
import zarr 
import h5py
from matplotlib import pyplot as plt

In [2]:
dataset_path = "data/pusht/pusht_cchi_v7_replay.zarr"
dataset_zarr = zarr.open(dataset_path, 'r')

for key in dataset_zarr.keys():
    print(key )

for key in dataset_zarr['meta']:
    print('meta:', key, dataset_zarr['meta'][key].shape)

for key in dataset_zarr['data'].keys():
    print('data:', key, dataset_zarr['data'][key].shape) 

data
meta
meta: episode_ends (206,)
data: action (25650, 2)
data: img (25650, 96, 96, 3)
data: keypoint (25650, 9, 2)
data: n_contacts (25650, 1)
data: state (25650, 5)


In [3]:
episode_ends=dataset_zarr['meta']['episode_ends'][:]
action=dataset_zarr['data']['action'][:]
img=dataset_zarr['data']['img'][:].astype(np.uint8)
keypoint=dataset_zarr['data']['keypoint'][:]
n_contacts=dataset_zarr['data']['n_contacts'][:]
state=dataset_zarr['data']['state'][:]

In [4]:
episode_starts = np.concatenate(([0], episode_ends[:-1]))

In [5]:
episode_starts[:10], episode_ends[:10]

(array([   0,  161,  279,  420,  579,  738,  895,  964, 1133, 1213]),
 array([ 161,  279,  420,  579,  738,  895,  964, 1133, 1213, 1347]))

In [6]:
def get_episode(episode_idx):
    start = episode_starts[episode_idx]
    end = episode_ends[episode_idx]
    actions=action[start:end]
    dactions = np.diff(actions, axis=0)
    dactions = np.concatenate([dactions, np.zeros((1, 2))], axis=0)
    return {
        'action': action[start:end],
        'daction': dactions,
        'img': img[start:end],
        'keypoint': keypoint[start:end],
        'n_contacts': n_contacts[start:end],
        'state': state[start:end],
        'agent_pos': state[start:end, :2],
    }

In [7]:
episode = get_episode(0)
episode['action'].shape, episode['daction'].shape, episode['img'].shape

((161, 2), (161, 2), (161, 96, 96, 3))

In [8]:
n_episode=len(episode_starts)
n_episode

206